In [36]:
import ee
import json
import geojson
import pandas_gbq as gbq
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
from shapely import wkt

In [37]:
service_account = 'austinbreunig@ee-obia-waterbodies.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-obia-waterbodies-33e289809d3b.json')
ee.Initialize(credentials)

In [38]:
sql = """
SELECT * FROM `ee-obia-waterbodies.ee_bq_test.ss_total_cat_depths`
"""

df = gbq.read_gbq(sql, project_id='ee-obia-waterbodies', dialect='standard')

Downloading: 100%|██████████|


In [39]:
df.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,"{""type"":""Polygon"",""coordinates"":[[[-95.4399074...",17056189,4,Very High,48039,0,5,10,15,18,33
1,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17056827,4,Very High,48039,0,5,10,15,18,33
2,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17057446,4,Very High,48039,0,5,10,15,19,34
3,"{""type"":""Polygon"",""coordinates"":[[[-95.4407407...",17057591,4,Very High,48039,0,5,10,15,19,34
4,"{""type"":""Polygon"",""coordinates"":[[[-95.4396296...",17057779,4,Very High,48039,0,5,10,15,19,34


In [40]:
def json_to_shapely(geometry):
    """
    Convert JSON geometry to WKT format.
    
    Args:
        json_geometry (dict): JSON geometry object.
    
    Returns:
        str: WKT representation of the geometry.
    """
    shapely_geometry = shape(json.loads(geometry))
    return shapely_geometry


In [41]:
gdf = df.copy()

In [42]:

gdf['geom'] = gdf['geom'].apply(json_to_shapely)

In [43]:
gdf['geom'].head()

0    POLYGON ((-95.43990740699996 28.96675925900007...
1    POLYGON ((-95.43740740699997 28.96787037000007...
2    POLYGON ((-95.43740740699997 28.96759259300006...
3    POLYGON ((-95.44074074099996 28.96703703700006...
4    POLYGON ((-95.43962962999994 28.96666666700002...
Name: geom, dtype: object

In [44]:
# shapely to wkt geom to geojson
gdf['geom'] = gdf['geom'].apply(lambda x: geojson.Feature(geometry=x))

In [45]:
gdf.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056189,4,Very High,48039,0,5,10,15,18,33
1,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056827,4,Very High,48039,0,5,10,15,18,33
2,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057446,4,Very High,48039,0,5,10,15,19,34
3,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057591,4,Very High,48039,0,5,10,15,19,34
4,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057779,4,Very High,48039,0,5,10,15,19,34


In [46]:
all_features = []
for index, row in gdf.iterrows():
    all_features.append(row['geom']['geometry']['coordinates'])

aoi = ee.Geometry.MultiPolygon(all_features)

In [47]:
col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [48]:
aoi_col = col.filterBounds(aoi)
image = aoi_col.filterDate('2023-06-01', '2023-06-30').first()

# ndvi calculation
ndvi = image.normalizedDifference(['B8', 'B4'])

# check if a dict is empty
if not ndvi.getInfo():
    print('empty')


In [49]:
band_df = pd.DataFrame()
for index, row in gdf.iterrows():
    coords = row['geom']['geometry']['coordinates']
    zone = ee.FeatureCollection(ee.Geometry.Polygon(coords))

    stats_dict = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=zone,
    scale=30
).getInfo()
    
    # create df from stats_dict
    df = pd.DataFrame(stats_dict, index=[0])
    band_df = pd.concat([band_df, df], axis=0, ignore_index=True)



    

In [50]:
band_df.head()

,nd
0,0.845340
1,0.826523
2,0.813726
3,0.816328
4,0.840232


In [51]:
gdf = gdf.join(band_df)


In [52]:
gdf.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_,nd
0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056189,4,Very High,48039,0,5,10,15,18,33,0.845340
1,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056827,4,Very High,48039,0,5,10,15,18,33,0.826523
2,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057446,4,Very High,48039,0,5,10,15,19,34,0.813726
3,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057591,4,Very High,48039,0,5,10,15,19,34,0.816328
4,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057779,4,Very High,48039,0,5,10,15,19,34,0.840232


In [207]:
gdf.columns

Index(['geom', 'ID', 'RISK_VALUE', 'SURGE_RISK', 'FIPSSTCO', 'CAT1DEP',
       'CAT2DEP', 'CAT3DEP', 'CAT4DEP', 'CAT5DEP', 'total_cat_', 'nd'],
      dtype='object')

In [208]:
gdf['geom'] = gdf['geom'].apply(lambda x: shape(x['geometry']))

In [183]:
gpd.GeoDataFrame(gdf, geometry='geom', crs=4326).to_file('ndvi.shp')

In [152]:
image

In [191]:
ndvi.getInfo()

{'type': 'Image',
 'bands': [{'id': 'nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [7741, 7881],
   'crs': 'EPSG:32615',
   'crs_transform': [30, 0, 174585, 0, -30, 3313215]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-95.882442745473, 29.916241781187402],
    [-95.88466594089704, 29.91619304537997],
    [-95.99455146212536, 29.476648225776238],
    [-96.10100454522356, 29.04777132195221],
    [-96.20674263451791, 28.61885317312932],
    [-96.31199552970635, 28.18906585924985],
    [-96.31065331966452, 28.188511353160738],
    [-95.53378332512604, 28.03584494684638],
    [-94.42392482234081, 27.809210544984897],
    [-94.39827630112785, 27.90679780616],
    [-94.3555008139391, 28.06928519217053],
    [-94.32005718871515, 28.203663466025045],
    [-94.27882232785078, 28.359612623380922],
    [-94.24452219386873, 28.48908590532632],
    [-94.17790059614084, 28.739864837933563],
   